[Link na zadatke](http://www.fer.unizg.hr/_download/repository/vjezba_2%5B4%5D.pdf)

In [85]:
import numpy as np
import math

norm = np.linalg.norm

### [Wrapper za brojanje fn. calls](https://stackoverflow.com/questions/1301735/counting-python-method-calls-within-another-method)

In [84]:
def counted(fn):
    def wrapper(*args, **kwargs):
        wrapper.called += 1
        return fn(*args, **kwargs)
    wrapper.called = 0
    wrapper.__name__= fn.__name__
    return wrapper

def counts(fn):
    def wrapper(f, *args, **kwargs):
        f.called = 0
        try:
            return fn(f, *args, **kwargs)
        finally:
            wrapper.calls_to_function = f.called
    wrapper.__name__= fn.__name__
    return wrapper

## Funkcije cilja

In [66]:
pt = lambda *x: np.array(x)
dim = lambda pt: len(pt)

@counted
def f1(x):
    x1, x2 = x
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2
f1_x0 = pt(-1.9, 2)
f1_xmin = pt(1, 1)
f1_min = 0

@counted
def f2(x):
    x1, x2 = x
    return (x1 - 4) ** 2 + 4 * (x2 - 2) ** 2
f2_x0 = pt(0.1, 0.3)
f2_xmin = pt(4, 2)
f2_min = 0

@counted
def f3(x):
    return np.sum(np.square(np.arange(1, x.shape[0] + 1) - x))
f3_x0 = pt(0, 0, 0, 0, 0)
f3_min = 0

@counted
def f4(x):
    x1, x2 = x
    return abs((x1 - x2) * (x1 + x2)) + math.sqrt(x1 ** 2 + x2 ** 2)
f4_x0 = pt(5.1, 1.1)
f4_min = 0

@counted
def f6(x):
    x = np.sum(np.square(x))
    return 0.5 + (math.sin(math.sqrt(x)) ** 2 - 0.5) / (1 + 0.001 * x) ** 2
f6_xmin = 0
f6_min = 0

### [Golden section](http://www.fer.unizg.hr/_download/repository/zlatni_rez.txt)

In [28]:
@counts
def zlatni_rez(f, a, b, e, k=0.5*(math.sqrt(5) - 1)):
    hi = b - k * (b - a)
    lo = a + k * (b - a)
    
    f_hi = f(hi)
    f_lo = f(lo)
    
    while (b - a) > e:
        if f_hi < f_lo:
            b = lo
            lo = hi
            hi = b - k * (b - a)
            f_lo = f_hi
            f_hi = f(hi)
        else:
            a = hi
            hi = lo
            lo = a + k * (b - a)
            f_hi = f_lo
            f_lo = f(lo)
    
    return (a + b) / 2

### [Unimodal](http://www.fer.unizg.hr/_download/repository/unimodalni.txt)

In [5]:
@counts
def unimodalni(f, pt, h=1):
    l, r = pt - h, pt + h
    m = pt
    step = 1
    
    fm = f(pt)
    fl = f(l)
    fr = f(r)
    
    if fm < fr and fm < fl:
        return l, r
    elif fm > fr:
        while True:
            l, m, fm = m, r, fr
            r = pt + h * (step * 2)
            fr = f(r)
            step *= 2
            if fm <= fr:
                return l, r
    else:
        while True:
            r, m, fm = m, l, fl
            l = pt - h * (step * 2)
            fl = f(l)
            step *= 2
            if fm <= fl:
                return l, r
    

### [Simplex](http://www.fer.unizg.hr/_download/repository/simplex.html)

In [92]:
@counts
def simplex_nelder_mead(
    f, x0, alpha=1, beta=0.5, gamma=2, sigma=0.5, epsilon=1e-6):
    
    x = simplex_pts(x0)
    reflect = lambda x0: x0 + alpha * (x0 - x[-1])
    expand = lambda xr: x0 + gamma * (xr - x0)
    stopping_condition = lambda xn, xb: xn - xb <= epsilon
    
    while True:
        xc = np.mean(x)
        xr = reflect()
        
        if f(xr) < f(x[l]):
            xe = expand(xr)
            if f(xe) < f(x[l]):
                x[h] = xe
            else:
                x[h] = xr
        else:
            fxr = f(xr)
            if all(fxr > f(xj) for j, xj in enumerate(x) if j != h):
                if fxr < f(x[h]):
                    x[h] = xr
                else:
                    pass # treba pomaknut sve tocke prema X[l] ??
            else:
                x[h] = xr
        
        if stopping_condition(x):
            break
            
    return x

### [Hooke-Jeeves](http://www.fer.unizg.hr/_download/repository/hj.html)

* x0 - pocetna tocka
* xB - bazna tocka 
* xP - pocetna tocka pretrazivanja
* xN - tocka dobivena pretrazivanjem

In [86]:
def reset_and_get_num_called(f):
    called = f.called
    f.called = 0
    return called

def lmap(f, x):
    return np.array(list(map(f, x)))

@counts
def hooke_jeeves(f, x_0, eps=1e-6):
    dx = 0.5 * np.ones_like(x_0)
    eps = eps * np.ones_like(x_0)

    xp = np.copy(x_0)
    xb = np.copy(x_0)

    while True:
        xn = explore(f, xp, dx)

        if f(xn) < f(xb):
            xp = 2 * xn - xb
            xb = xn
        else:
            dx /= 2
            xp = xb
            if np.any(eps > dx):
                break
    return xp
    #return xp, reset_and_get_num_called(f)

def explore(f, xp, dx):
    x = np.copy(xp)
    for i in range(dim(xp)):
        P = f(x)
        x[i] = x[i] + dx[i]
        if f(x) > P:
            x[i] = x[i] - 2 * dx[i]
            if f(x) > P:
                x[i] = x[i] + dx[i]
    return x

In [87]:
hooke_jeeves(f4, pt(10, 0))

array([10,  0])

In [88]:
hooke_jeeves.calls_to_function

114

# Zadaci
## 1.
Definirajte jednodimenzijsku funkciju br. 3, koja će imati minimum u točki 3. Kao početnu točku
pretraživanja postavite točku 10. Primijenite sva tri postupka na rješavanje ove funkcije te ispišite
pronađeni minimum i broj evaluacija funkcije za svaki pojedini postupak. Probajte sve više
udaljavati početnu točku od minimuma i probajte ponovo pokrenuti navedene postupke. Što možete
zaključiti? 

In [99]:
import pandas as pd

hooke_jeeves(f3, pt(10))
hooke_jeeves.calls_to_function

data = []
for f in [ hooke_jeeves ]:
    result = f(f3, pt(10))
    data.append([f.__name__, f.calls_to_function, result])

pd.DataFrame(data, columns=['Algoritam', 'Broj poziva', 'Rezultat'])

,Algoritam,Broj poziva,Rezultat
0,hooke_jeeves,76,[10]


## 2.
Primijenite simpleks po Nelderu i Meadu, Hooke-Jeeves postupak te pretraživanje po koordinatnim
osima na funkcije 1-4 uz zadane parametre i početne točke (broj varijabli funkcije 3 najmanje 5). Za
svaki postupak i svaku funkciju odredite minimum koji su postupci pronašli i potrebni broj
evaluacija funkcije cilja koji je potreban do konvergencije (prikažite tablično). Što možete zaključiti
iz rezultata?

In [101]:
fs = [f1, f2, f3, f4]
x0s = [f1_x0, f2_x0, f3_x0, f4_x0]

data = []
for search_f in [hooke_jeeves]:
    for f, x0 in zip(fs, x0s):
        result = search_f(f, x0)
        data.append(
            [search_f.__name__, search_f.calls_to_function, result])

pd.DataFrame(data, columns=['Algoritam', 'Broj poziva', 'Rezultat'])

,Algoritam,Broj poziva,Rezultat
0,hooke_jeeves,694,"[1.00000152588, 1.0000038147]"
1,hooke_jeeves,305,"[3.99999961853, 2.00000076294]"
2,hooke_jeeves,228,"[0, 0, 0, 0, 0]"
3,hooke_jeeves,172,"[3.1, 3.1]"


## 3.
Primijenite postupak Hooke-Jeeves i simpleks po Nelderu i Meadu na funkciju 4 uz početnu točku
(5, 5). Objasnite rezultate!

In [102]:
hooke_jeeves(f4, pt(5, 5))

array([5, 5])

## 4.
Primijenite simpleks po Nelderu i Meadu na funkciju 1. Kao početnu točku postavite točku (0.5,0.5).
Provedite postupak s nekoliko različitih koraka za generiranje početnog simpleksa (primjerice iz
intervala od 1 do 20) i zabilježite potreban broj evaluacija funkcije cilja i pronađene točke
minimuma. Potom probajte kao početnu točku postaviti točku (20,20) i ponovo provesti eksperiment.
Što možete zaključiti?

## 5.
Primijenite jedan postupak optimizacije na funkciju 6 u dvije dimenzije, tako da postupak pokrećete
više puta iz slučajno odabrane početne točke u intervalu [-50,50]. Možete li odrediti vjerojatnost
pronalaženja globalnog optimuma na ovaj način? (smatramo da je algoritam locirao globalni
minimum ako je nađena vrijednost funkcije cilja manja od 10^-4)

In [107]:
ncalls = 20

data = []
for i in range(ncalls):
    random = np.random.randint(-50, 50 + 1, size=2)
    result = hooke_jeeves(f6, pt(*random))
    data.append([hooke_jeeves.__name__, 
                 random,
                 hooke_jeeves.calls_to_function, 
                 result])

pd.DataFrame(data, columns=['Algoritam', 'x0', 'Broj poziva', 'Rezultat'])

,Algoritam,x0,Broj poziva,Rezultat
0,hooke_jeeves,"[-30, 4]",146,"[-28, 4]"
1,hooke_jeeves,"[33, 24]",114,"[33, 24]"
2,hooke_jeeves,"[13, 18]",114,"[13, 18]"
3,hooke_jeeves,"[34, 31]",114,"[34, 31]"
4,hooke_jeeves,"[-19, 40]",139,"[-18, 40]"
5,hooke_jeeves,"[12, 20]",114,"[12, 20]"
6,hooke_jeeves,"[-3, 9]",133,"[-3, 9]"
7,hooke_jeeves,"[29, -24]",133,"[29, -24]"
8,hooke_jeeves,"[-19, -25]",134,"[-19, -25]"
9,hooke_jeeves,"[13, 13]",114,"[13, 13]"


# Matlab
* `fminbnd` (1 var)
* `fminsearch` (vise var; simplex po N&M)
* `fminunc` (vise var; razliciti alg.)
* `fmincon` (vise var, uz ogranicenja)